In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

/Users/sampathweb/miniconda3/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [8]:
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("data/hacknight_train.csv")

In [3]:
test_df = pd.read_csv("data/hacknight_test.csv")

In [4]:
train_df.head()

,week,homescore,awayscore,Hm_CS_wins,Hm_CS_points,Hm_CS_passatt,Hm_CS_passcomp,Hm_CS_passyds,Hm_CS_passtds,Hm_CS_passtwopta,...,AwA_PS_totalrecfumbs,AwA_PS_fumbslost,AwA_PS_sacks,AwA_PS_tackles,AwA_PS_kickrets,AwA_PS_kickretyds,AwA_PS_kickrettds,AwA_PS_puntrets,AwA_PS_puntretyds,AwA_PS_puntrettds
0,7,31,21,0.833333,27.500000,31.833333,21.833333,251.666667,1.833333,0.000000,...,1.6250,0.7500,2.5000,41.6875,2.0625,44.750,0.0000,3.0000,39.8750,0.1875
1,5,16,14,0.333333,25.666667,40.000000,27.333333,301.333333,2.666667,0.333333,...,1.3125,0.5625,2.0625,43.3125,3.2500,72.625,0.0000,1.9375,10.7500,0.0000
2,8,24,30,0.500000,21.333333,39.666667,23.666667,266.000000,2.000000,0.500000,...,1.0000,0.6875,2.0625,40.5000,2.1875,49.500,0.0625,1.8750,12.0625,0.0000
3,15,27,28,0.538462,22.307692,31.923077,18.384615,238.153846,1.230769,0.153846,...,1.4375,0.9375,1.6875,43.1250,4.5625,91.000,0.0625,3.3750,38.2500,0.0625
4,17,27,9,0.800000,26.000000,26.466667,16.800000,222.400000,1.733333,0.066667,...,1.1250,0.2500,2.1250,42.3750,1.8750,44.875,0.0000,2.5625,24.3125,0.0000


In [26]:
X = train_df.iloc[:, 3:]
y1 = train_df["homescore"].values.astype(np.float32)
y2 = train_df[["awayscore"]]

In [53]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.1)

In [54]:
X_mean = X_train.mean()

In [60]:
X_train = X_train.fillna(X_mean, inplace=True)
X_test = X_test.fillna(X_mean, inplace=True)

/Users/sampathweb/miniconda3/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [28]:
from sklearn.ensemble import RandomForestRegressor

In [61]:
model_y1 = RandomForestRegressor(n_estimators=10, min_samples_leaf=5, criterion="mae", n_jobs=-1)
model_y2 = RandomForestRegressor(n_estimators=10, min_samples_leaf=5, criterion="mae", n_jobs=-1)

In [62]:
model_y1.fit(X_train, y1_train)
model_y2.fit(X_train, y2_train)

/Users/sampathweb/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=5,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [63]:
model_y1.score(X_test, y1_test)

0.0075355929242738586

In [64]:
model_y2.score(X_test, y2_test)

-0.078744944865452249

In [68]:
X_new = test_df.iloc[:, 2:]

In [71]:
X_new = X_new.fillna(X_mean)

In [73]:
y1_pred = model_y1.predict(X_new)
y2_pred = model_y2.predict(X_new)

In [74]:
y1_pred = np.round(y1_pred).astype(int)
y2_pred = np.round(y2_pred).astype(int)

In [83]:
pd.DataFrame({
    "index": np.arange(len(y1_pred)),
    "homescore": y1_pred,
    "awayscore": y2_pred
})[["index", "homescore", "awayscore"]].to_csv("submission1.csv", index=False)

In [84]:
!head submission1.csv

index,homescore,awayscore
0,27,26
1,28,23
2,18,15
3,23,24
4,16,20
5,30,23
6,26,22
7,25,26
8,21,15
